# Trendyol Vegan Candle Scaper

In [22]:
# Libraries

import requests
import json
import math
import time

import pandas as pd
from datetime import date

# API Endpoints

API endpoints are found using search page's infinite scroll capability.

In the 1st page all details will be collected and then total page count will be calculated. Scraping will continue to the last page afterwards.

In [2]:
url = "https://public.trendyol.com/discovery-web-searchgw-service/v2/api/infinite-scroll/" \
    "sr?q=vegan+kokulu+mum&qt=vegan+kokulu+mum" \
    "&st=vegan+kokulu+mum&os=1" \
    "&pi=1" \
    "&culture=tr-TR" \
    "&pId=0" \
    "&scoringAlgorithmId=2"

ty_vc = requests.get(url)

raw_data = json.loads(ty_vc.text)

page_count = math.ceil(raw_data["result"]["totalCount"] / 24)

# Collecting 1st Page Items

In [3]:
# pd.set_option('display.max_columns', None)
collected_items = pd.json_normalize(raw_data["result"]["products"])

# Collecting Other Pages

In [19]:
for page in range(3, page_count):
    url_inner = "https://public.trendyol.com/discovery-web-searchgw-service/v2/api/infinite-scroll/" \
        "sr?q=vegan+kokulu+mum&qt=vegan+kokulu+mum" \
        "&st=vegan+kokulu+mum&os=1" \
        f"&pi={page}" \
        "&culture=tr-TR" \
        "&pId=0" \
        "&scoringAlgorithmId=2"
    
    ty_vc_inner = requests.get(url_inner)
    
    raw_data_inner = json.loads(ty_vc_inner.text)
    collected_items = pd.concat([collected_items, pd.json_normalize(raw_data_inner["result"]["products"])], axis=0)

    time.sleep(1.2)

    print(f"{page} is completed")

In [21]:
collected_items.reset_index(inplace=True)

In [23]:
collected_items.to_csv(f"ty_candle_export_{date.today()}.csv")

# QA Playground

In [50]:
# pd.merge(collected_items, pd.json_normalize(collected_items.promotions), left_index=True, right_index=True)

# inner_df = pd.json_normalize(collected_items.promotions)
# collected_items.promotions

In [60]:
# inner_df[1][4]["name"]

"180 TL'ye %10 İndirim"

In [ ]:
pd.json_normalize(
    raw_data['result']['products'],
    record_path=['sections'],
    meta=['id', 'name', 'images', 'imageAlt', 'brand.id', 'brand.name', 'tax', 'businessUnit',
        'showSexualContent', 'productGroupId', 'hasReviewPhoto', 'cardType', 'hasCollectableCoupon',
        'sameDayShipping', 'isLegalRequirementConfirmed', 'badges'],
    errors='ignore',
    record_prefix="_"
    )